# Test the self hosted service 
The service is set up with docker. To find out how to change the possibilities what to control the possible routing parameters from the outside in this notbook a call for the service is implemented. 

In [16]:
import pandas as pd
from routingpy import ORS, Google
from routingpy.exceptions import RouterApiError
import folium
import requests
import json

In [13]:
class RoutingTool: 
    """Calculate different routes for each given start and end point."""
    def __init__(self):
        '''Define url.'''
        self.base_url_ors = 'http://localhost:8080/ors/v2/directions'
        self.setting_ors_cargo_bike()
        
    def setting_ors_cargo_bike(self, elevation = True, avoid_features = ["junction", "steps"], steepness_difficulty = 0):

        ors_cargo_bike_green_args = {
            'elevation': elevation, # take elevation into account
            'language': 'de',
            'options': {
                "avoid_features": avoid_features,# avoid steps
                'profile_params': {
                    'weightings': {
                        'steepness_difficulty': steepness_difficulty, # Beginner steepness difficulty
                        'green': 1
                    },
                },
            },
            'extra_info': [
                'waytype',
                'surface'
            ],
        }     
        ors_cargo_bike_args = {
            'elevation': elevation, # take elevation into account
            'language': 'de',
            'options': {
                'avoid_features': avoid_features,# avoid steps
                'profile_params': {
                    'weightings': {
                        'steepness_difficulty': steepness_difficulty, # Beginner steepness difficulty
                    },
                },
            },
            'extra_info': [
                'waytype',
                'surface'
            ],
        }   
        self.args = [ors_cargo_bike_green_args, ors_cargo_bike_args]

    def calculate_directions(self, coordinates):
        routes = []
        client, profile, args, name = ORS(base_url = self.base_url_ors), 'cycling-cargo', self.args, 'ORS'
        try:
            for arg in args:
                print("Trying directions: ", name)
                route = client.directions(locations=coordinates, profile=profile, **arg)
                print(route)
                routes.append((route, name))
        except RouterApiError as rae:
            # print("Error", rae.message['error'])
            # self.rae = rae
            # return False
            print(rae)
        except KeyError as ke:
            # print("Error", ke.message['error'])
            # return False
            print(ke)
        self.routes = routes
        return True
    def direction_wgs84(self):
        '''Return the coordinates in wgs84.'''
        return self.routes
    def direction_etrs89(self):
        '''Return the coordinates in etrs89.'''
        try:
            ors_coords = [self.convert_coord_systems(geom) for geom in self.routes[0][0].geometry]
        except:
            return False
        etrs89  = ors_coords
        return etrs89
        


In [14]:
routes = []
coord = [[8.67710307065867, 49.4229726228998], [8.683560278482277, 49.39729589125126]] 
tool_track = RoutingTool()
if tool_track.calculate_directions(coord):
    print(tool_track.routes)
    routes.append(tool_track.routes[0][0])
    routes.append(tool_track.routes[1][0])

Trying directions:  ORS
'content-type'
[]


# Alternative approach to calling local ORS

In [17]:
check_url = 'http://localhost:8080/ors/v2/health'

check_response = requests.get(check_url)
ideal_response = {"status":"ready"}

if check_response.json() == ideal_response:

    url = 'http://localhost:8080/ors/v2/directions/cycling-cargo'

    headers = {
    'Content-Type': 'application/json; charset=utf-8',
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8'
    }   

    data_1 = {
        "coordinates":[[8.67710307065867, 49.4229726228998], [8.683560278482277, 49.39729589125126]],
        "options": {
            "avoid_features":["junction", "steps"]
        },
        "attributes":["avgspeed"],
        "instructions":"false",
        "preference":"shortest",
        "units":"m"
    }

    data_2 = {
    "coordinates":[[8.67710307065867, 49.4229726228998], [8.683560278482277, 49.39729589125126]],
    "options": {
        "avoid_features":["steps"]
    },
    "attributes":["avgspeed"],
    "instructions":"false",
    "preference":"shortest",
    "units":"m"
    }

    response_1 = requests.post(url, headers=headers, data=json.dumps(data_1))

    response_2 = requests.post(url, headers=headers, data=json.dumps(data_2))

    # Remove comments below to save file

    # filename = 'response-custom.json'

    # with open(filename, 'w') as f:
    #     json.dump(response.json(), f, indent=4)

else:
    print("ORS not ready...")
    print("Response: ", check_response.json())

In [24]:
response = response.json()

In [25]:
class DisplayRoute():
    def swap_lat_lon(self,coordinates):
        return list(map(lambda point: [point[1], point[0]], coordinates))
    def display_map(self,routes):
        results = self.setting_up_api(routes)
        my_map = folium.Map()

        for route, name, color in results:
            points = self.swap_lat_lon(route.geometry) 
            feature_group = folium.FeatureGroup(name=name)
            folium.PolyLine(points, color=color, opacity=0.5, tooltip=name).add_to(feature_group)
            feature_group.add_to(my_map)

        folium.LayerControl(collapsed=False, hideSingleBase=True).add_to(my_map)
        my_map.fit_bounds(my_map.get_bounds())
        return my_map
    def setting_up_api(self, routes):
        settings = []
        settings.append(routes)
        settings.append((routes[0], 'Green', 'green'))
        settings.append((routes[1], ' Not Green', 'blue'))
        return settings

In [ ]:
map_routes = DisplayRoute()
map_routes.display_map(response)

# Alternative way to display routes

In [ ]:
import folium
import polyline


# Decode polyline
geometry = response['routes'][0]['geometry']
decoded = polyline.decode(geometry)

# Create a map centered at the average latitude and longitude of your route
map_center = [sum(lat for lat, _ in decoded) / len(decoded), sum(lon for _, lon in decoded) / len(decoded)]
route_map = folium.Map(location=map_center, zoom_start=14)

# Add the decoded polyline to the map
path = folium.PolyLine(locations=decoded, color='blue', weight=2.5, opacity=1).add_to(route_map)

# Display the map
route_map
